In [1]:
import pandas as pd
import numpy as np
from influxdb import DataFrameClient
from dotenv import load_dotenv
import os
from fnmatch import fnmatch

In [2]:
load_dotenv()

True

In [3]:
user = os.getenv('DB_USER')
password = os.getenv('DB_PWD')
host = os.getenv('DB_ADDR')
port = os.getenv('DB_PORT')
dbname = 'GasPriceMonitorization'
protocol = 'line'
client = DataFrameClient(host, port, user, password, dbname)

In [4]:
measurement_gas = 'GasPrice'
measurement_crude = 'CrudeOil'

In [5]:
def get_all_files_matching_pattern(root: str, pattern: str) -> list[str]:
    paths = []

    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                paths.append(os.path.join(path, name))
    
    return paths

In [ ]:
def upload_all_gas():
    root = 'archived_data'
    pattern = "gas_tmp-*.json"

    paths = get_all_files_matching_pattern(root, pattern)

    for path in paths:
        data = pd.read_json(path, orient='records')
        data.set_index('Timestamp', inplace=True)
        data.dropna(inplace=True)

        columns = data.columns
        columns = columns.drop(['Price'])

        client.write_points(data, measurement_gas, protocol=protocol, field_columns=['Price'], tag_columns=columns)

In [ ]:
def upload_all_crude():
    root = 'archived_data'
    pattern = "crude_tmp-*.json"

    paths = get_all_files_matching_pattern(root, pattern)

    for path in paths:
        data = pd.read_json(path, orient='records')
        data.set_index('Date', inplace=True)
        data.dropna(inplace=True)

        columns = data.columns

        client.write_points(data, measurement_crude, protocol=protocol, field_columns=columns)

In [ ]:
upload_all_gas()
upload_all_crude()